# Satellite Location Demo

In [1]:
import micropip
await micropip.install("https://ds-wheels.s3.amazonaws.com/sgp4-2.23-cp312-cp312-pyodide_2024_0_wasm32.whl")

from sgp4.api import accelerated
print(accelerated)

True


Timeliness is an important aspect for countless uses of Earth Observation (EO) data. Humanitarians and emergency response organizations, for example, benefit from imagery immediately before or after an emergency.

Knowing when an image was captured of an area of interested, or when it will be captured next, can help humanitarians emergency response organizations better understand where aid is most needed.

This notebook demonstrates how the location of EO satellites can be computed in an area of interest. This can be used as a proxy of where images were and will be captured. 

These techniques could be combined with those in other notebooks in this repository, such as loading STAC items and calculating how a disaster impacts people, to create automated analysis for past and future disasters.

## Load Libraries and Data

We'll use the [sgp4](https://pypi.org/project/sgp4/) and [skyfield](https://rhodesmill.org/skyfield/) libraries, which help us calculate the position of satellites using complex orbital physics. Shapely will help us with some of the geometry creation and calculations, and Lonboard will be used to visualize the results.

In [2]:
import micropip

In [3]:
%pip install pyodide-unix-timezones

In [4]:
deps = [
    "https://ds-wheels.s3.amazonaws.com/pyarrow-17.0.0-cp312-cp312-pyodide_2024_0_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_core-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_compute-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_io-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/geoarrow_rust_core-0.3.0b1-cp38-abi3-emscripten_3_1_58_wasm32.whl",
    "palettable",
    "matplotlib",
    "lonboard==0.10.0b2"
]
await micropip.install(deps)

In [5]:
%pip install skyfield

In [6]:
from skyfield.api import load, wgs84, Timescale
from skyfield.jpllib import SpiceKernel

from datetime import datetime, timedelta, timezone

from lonboard import viz, Map, ScatterplotLayer, basemap, PathLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Oranges_9

import geopandas as gpd
import pandas as pd
from shapely import Point, LineString, Polygon

import requests
import pyarrow

## Calculating the position of a satellite

We'll load information about specific satellites from [Celestrak](https://celestrak.org). Due to the complexities of orbital physics, data about satellites needs to be updated frequently is only accurate for about 1 week before and after "epoch".

For this example, we'll calculate the position of Sentinel 2b.

In [7]:
sentinel_2b_url = "https://celestrak.org/NORAD/elements/gp.php?CATNR=42063"

r = requests.get(sentinel_2b_url)

with open('out1.txt', 'wb') as output:
    output.write(r.content)

In [8]:
sentinel_2b = load.tle_file("out1.txt", reload=True, filename="sentinel_2b")

In [9]:
# Check if the data loaded properly

satellite = sentinel_2b[0]
satellite

<EarthSatellite SENTINEL-2B catalog #42063 epoch 2024-10-01 05:18:58 UTC>

For sudden-onset emergencies, the time immediately before an after an event are the most critical. We can set a time range of +2 days (48h) from a certain point in time. For the purposes of this notebook, we'll use `now` (the time when the cell is run). These techniques can also be applied in the past, if a before/after analysis is needed.

In [10]:
# WIP

# f = open("../data/de421.bsp", "rb")
# ephemeris = f.read()

# eph = SpiceKernel(ephemeris)

In [11]:
# Calculate the current UTC time (without microseconds), then creating a time range + 48h

# load timescale
ts = load.timescale()

# load "ephemeris" (https://rhodesmill.org/skyfield/api.html#planetary-ephemerides), which let's us determine if a satellite is illuminated by the sun (as a proxy of if the ground is lit)
# eph = open('data/de421.bsp')

now = datetime.now(timezone.utc).replace(microsecond=0)
t1 = now
t2 = now + timedelta(days=2)

To calculate the position of the satellite throughout the time range, we can initiate a timer and then calculate the position at each time step (defined below).

In [12]:
# initiate

timer = t1

df = pd.DataFrame(columns=['satellite', 'timestamp', 'coordinates', 'lng', 'lat', 'daytime'])

rows = []

The `location_iteration` function lodes the geocentric location, calculates the latitude and longitude, converts them to decimal degrees, and saves them as a point coordinate. It also checks if the satellite is sunlit at the time of calculation, which can be used as an approximation of if it is daytime below.

In [13]:
def location_iteration(timer, sat):
    geocentric = sat.at(Timescale.from_datetime(ts, timer))
    lat, lon = wgs84.latlon_of(geocentric)
    longitude = lon.degrees
    latitude = lat.degrees
    # daytime = geocentric.is_sunlit(eph)
    coords = Point(longitude, latitude)

    return timer, longitude, latitude, coords

Then, we can iterate over the time frame. The more frequent the measurements, the slower the calculation takes.

In [14]:
while timer <= t2:
    timer, longitude, latitude, coords = location_iteration(timer, sentinel_2b[0])

    row = pd.DataFrame({'satellite': satellite.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude}, index=[0])
    rows.append(row)    

    timer += timedelta(seconds=15) # ~13 seconds

df = pd.concat(rows, ignore_index=True)
df["time_string"] = df["timestamp"].dt.strftime('%Y-%m-%d %X')

We can then save this as a geodataframe.

In [15]:
sentinel_2b_gdf = gpd.GeoDataFrame(df, geometry="coordinates")
sentinel_2b_gdf

,satellite,timestamp,coordinates,lng,lat,time_string
0,SENTINEL-2B,2024-10-01 13:49:03+00:00,POINT (126.53351 23.38691),126.533511,23.386907,2024-10-01 13:49:03
1,SENTINEL-2B,2024-10-01 13:49:18+00:00,POINT (126.31187 24.27323),126.311870,24.273228,2024-10-01 13:49:18
2,SENTINEL-2B,2024-10-01 13:49:33+00:00,POINT (126.08801 25.15928),126.088012,25.159282,2024-10-01 13:49:33
3,SENTINEL-2B,2024-10-01 13:49:48+00:00,POINT (125.86181 26.04506),125.861811,26.045056,2024-10-01 13:49:48
4,SENTINEL-2B,2024-10-01 13:50:03+00:00,POINT (125.63314 26.93054),125.633137,26.930538,2024-10-01 13:50:03
...,...,...,...,...,...,...
11516,SENTINEL-2B,2024-10-03 13:48:03+00:00,POINT (-62.03549 -55.36913),-62.035490,-55.369128,2024-10-03 13:48:03
11517,SENTINEL-2B,2024-10-03 13:48:18+00:00,POINT (-62.51561 -56.22753),-62.515613,-56.227534,2024-10-03 13:48:18
11518,SENTINEL-2B,2024-10-03 13:48:33+00:00,POINT (-63.01478 -57.08431),-63.014778,-57.084307,2024-10-03 13:48:33
11519,SENTINEL-2B,2024-10-03 13:48:48+00:00,POINT (-63.53449 -57.93933),-63.534488,-57.939331,2024-10-03 13:48:48


Let's refine our area of interest to see when/where satellites pass overhead. As the 2024 hurricane season in the Americas is under way, we can focus on the US state of Florida for our analysis.

In [16]:
min_lon = x_min = -89
max_lon = x_max = -74
min_lat = y_min = 22
max_lat = y_max = 32

bbox=[min_lon, max_lon, min_lat, max_lat]
bbox

# Defining our AOI as a polygon shape
florida = Polygon([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max), (x_min,y_min)])

In [17]:
mask_lon = (sentinel_2b_gdf.lng >= min_lon) & (sentinel_2b_gdf.lng <= max_lon)
mask_lat = (sentinel_2b_gdf.lat >= min_lat) & (sentinel_2b_gdf.lat <= max_lat)

aoi = sentinel_2b_gdf.where(mask_lon & mask_lat).dropna()

#sentinel_2b_daylit = aoi[aoi["daytime"] == True]
#sentinel_2b_daylit

Next, we can visualize these points on a map. They are color coded using a continuous color scale, with white being `now` and darker purple being further after `now`.

In [18]:
# This creates a range from 0-1 to define our colormap.
time_norm = (aoi.timestamp - t1) / (t2 - t1)

colors = apply_continuous_cmap(time_norm, Oranges_9)

In [19]:
layer = ScatterplotLayer.from_geopandas(
    aoi,
    # extensions=[filter_extension],
    get_fill_color=colors,
    radius_min_pixels = 3
    # get_filter_value=filter_values,
    # filter_range=initial_filter_range,
)

m = Map(
    layer,
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m

/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

We can clearly see the passes that Sentinel 2b takes in our area of interest and during our time frame, and during daytime hours.

---

## Calculating the position of a satellite constellation

Usually, humanitarians or emergency response organizations aren't concerned about which satellite captures an image, as long as they have a timely image with the appropriate resolution, (lack of) cloud cover, bands, etc.

The following example calculates the position of Planet satellites. With their portfolio's high spatial and temporal resolution, they are able to capture high-resolution images immediately before or after an event.

In [20]:
planet_url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=planet&FORMAT=tle"

r = requests.get(planet_url)

with open('out2.txt', 'wb') as output:
    output.write(r.content)

In [21]:
planet_swarm = load.tle_file("out2.txt", reload=True, filename="planet_swarm")
planet_swarm[0]

<EarthSatellite SKYSAT-A catalog #39418 epoch 2024-10-01 03:37:32 UTC>

We then calculate the locations for all Planet satellites during our time frame in set increments.

In [22]:
timer = t1

rows = []

while timer <= t2:
    for sat in planet_swarm:
        timer, longitude, latitude, coords = location_iteration(timer, sat)

        row = pd.DataFrame({'satellite': sat.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude}, index=[0])
        rows.append(row)    

    timer += timedelta(minutes=2) # ~ 3.5 min

planet_swarm_df = pd.concat(rows, ignore_index=True)
planet_swarm_df["time_string"] = planet_swarm_df["timestamp"].dt.strftime('%Y-%m-%d %X')


In [23]:
planet_swarm_gdf = gpd.GeoDataFrame(planet_swarm_df, geometry="coordinates")

Again, we want to look at the satellites in our AOI. Because Planet satellites capture optical images, the following shows the next passes that occur when the satellite is lit (as a proxy of when the ground below is lit).

In [24]:
mask_lon = (planet_swarm_gdf.lng >= min_lon) & (planet_swarm_gdf.lng <= max_lon)
mask_lat = (planet_swarm_gdf.lat >= min_lat) & (planet_swarm_gdf.lat <= max_lat)

aoi_planet = planet_swarm_gdf.where(mask_lon & mask_lat).dropna()
# aoi_planet_day = aoi_planet[aoi_planet.daytime == True]
# aoi_planet_day[aoi_planet_day["timestamp"] > now]

# planet_day = planet_swarm_gdf[planet_swarm_gdf.daytime == True]
# planet_day[planet_day["timestamp"] > now]

## Visualize path of satellites

Instead of visualizing points on a map, a better representation would be a line. We can "connect the dots" calculated in the previous step.

In [25]:
def create_individual_linestrings(coords, timestamps, time_strings):
    coords_list = coords.tolist()
    timestamps_list = timestamps.tolist()
    time_strings_list = time_strings.tolist()
    line_segments = [
        {'linestring': LineString([coords_list[i], coords_list[i + 1]]), 'timestamp': timestamps_list[i], 'time_string': time_strings_list[i]} 
        for i in range(len(coords_list) - 1)
    ]
    return line_segments

In [26]:
new_rows = []

for satellite, group in aoi_planet.groupby('satellite'):
    line_segments = create_individual_linestrings(group['coordinates'], group['timestamp'], group['time_string'])
    for segment in line_segments:
        new_rows.append({'satellite': satellite, 'linestring': segment['linestring'], 'timestamp': segment['timestamp'], 'time_string': segment['time_string']})

path_segments = gpd.GeoDataFrame(new_rows, geometry='linestring')
path_segments

,satellite,linestring,timestamp,time_string
0,FLOCK 4BE-1,"LINESTRING (-88.15946 30.57479, -74.70960 29.1...",2024-10-01 16:49:03+00:00,2024-10-01 16:49:03
1,FLOCK 4BE-1,"LINESTRING (-74.70960 29.12269, -83.72621 30.1...",2024-10-02 03:21:03+00:00,2024-10-02 03:21:03
2,FLOCK 4BE-1,"LINESTRING (-83.72621 30.18979, -85.45370 22.6...",2024-10-02 16:31:03+00:00,2024-10-02 16:31:03
3,FLOCK 4BE-10,"LINESTRING (-82.20151 28.85105, -77.32728 26.5...",2024-10-02 03:51:03+00:00,2024-10-02 03:51:03
4,FLOCK 4BE-11,"LINESTRING (-82.81842 29.26100, -77.94482 27.0...",2024-10-02 03:53:03+00:00,2024-10-02 03:53:03
...,...,...,...,...
282,SKYSAT-C8,"LINESTRING (-80.69816 28.72227, -88.08049 24.3...",2024-10-01 19:37:03+00:00,2024-10-01 19:37:03
283,SKYSAT-C8,"LINESTRING (-88.08049 24.32681, -81.08370 24.4...",2024-10-02 07:37:03+00:00,2024-10-02 07:37:03
284,SKYSAT-C9,"LINESTRING (-87.97592 30.45809, -81.26211 28.7...",2024-10-01 20:03:03+00:00,2024-10-01 20:03:03
285,SKYSAT-C9,"LINESTRING (-81.26211 28.73325, -88.81966 25.4...",2024-10-02 19:35:03+00:00,2024-10-02 19:35:03


If these line segments are not within the AOI, we can drop them from the dataframe.

In [27]:
def is_within_aoi(linestring, florida):
    return linestring.intersects(florida)

In [28]:
path_segments['within_aoi'] = path_segments['linestring'].apply(lambda x: is_within_aoi(x, florida))
path_segments_clipped = path_segments[path_segments['within_aoi']].drop(columns='within_aoi')

path_segments_clipped['length'] = path_segments_clipped['linestring'].apply(lambda x: x.length)

# This is a way to drop segments that are not proper representations of paths, such as errors caused by traversing the International Date Line, or jumps between calculated points when a satellite enters and exits a nighttime area.
path_segments_clipped = path_segments_clipped[path_segments_clipped["length"] <= 75].drop(columns='length')

As with the single satellite, we can create a color scale that shows the time of the satellite's pass for each line segment. In this example, white is closer to `now` and orange is closer to +48h.

In [29]:
# This creates a range from 0-1 to define our colormap.
time_norm_planet_swarm = (path_segments_clipped.timestamp - t1) / (t2 - t1)

colors_planet_swarm = apply_continuous_cmap(time_norm_planet_swarm, Oranges_9, alpha=.5)

We can roughly estimate the width of the ground coverage of a Planet satellite to be 6km (i.e., the width of an image is 6km). To visualize this, we can set `get_width` to be 6km.

In [30]:
layer2 = PathLayer.from_geopandas(
    path_segments_clipped,
    get_color = colors_planet_swarm,
    get_width=6000,
    opacity=1,
    auto_highlight=True
)

/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [32]:
m2 = Map(
    [layer2],
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m2

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

As expected, there are many more passes than by a single satellite. Due to small area of interests for many events, cloud cover, and other variables, a larger constellation lets practitioners then narrow down their search.

The visualization of these paths give practitioners a good sense of the time an image of an area of interest will be captured. For example, a practitioner would be interested in an AOI of this size, but might be particularly interested in images of Miami. They could zoom in to Miami and see if a path covers or comes close to Miami. If so, and if there isn't excessive cloud cover, an image could be made available or tasked. If multiple paths cover Miami, that indicates a better chance of capturing a valuable image.

Once a satellite is calculated to have passed over an AOI, the next step could be to analyze the imagery, ideally using the other STAC and COG focused notebooks in this repository. Combining a positioning calculation with automated image loading and processing could be the foundations for a powerful EO-based monitoring and alerting system.